## **갤러리아 보고 (2022.10.06)**

---

### **분석 목적** 
* 타겟 고객군의 등급 상향 이동 여부 분류 데이터 분석 
* 신규 고객 온보딩 분석 
* 향후 우수고객 전환 관련 데이터 분석 (1번과 중복되는 것 같음)
* 고객 이탈분석 및 이탈관련 index 화 (고객 프로파일링)
* look-alike 타겟팅 (갤러리아 외 고객의 데이터 받을 수 있는가?)


### **1. 갤러리아 데이터 불러오기**
* 1-1) data read 하기
* 1-2) 데이터 결합 및 안쓰는 컬럼 drop 하기

### **2. EDA**

* 2-1) feature engineering  
* 2-2) Hypothesis and test

### **3. 데이터 핸들링**

* 3-1) Missing Imputation 
* 3-2) Feature selection 

### **4. 모형 해석**
* 3-1) Paramteric model
* 3-2) Assumption check  

### **5. Machine Laerning**
* 4-1) Ensemble 
* 4-2) Neural Net 
* 4-3) Prediction Performance Result


### **6. Reference** 
* ref 


----


In [2]:
import os 
import sys 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import re 
from pandas_profiling import ProfileReport
import openpyxl
import plotly.offline as plyo
import plotly.graph_objects as go
import cufflinks
import warnings


---
* 분석 환경설정 


In [5]:
# Jupyter 환경 설정 

warnings.filterwarnings(action='ignore') 
plt.rcParams['font.family'] = 'AppleGothic'
file_path = '/Users/dk/Downloads/scheme'
os.chdir(file_path)


---

### **1. 갤러리아 데이터 불러오기**
* 1-1) data read 하기 


In [6]:
file_nm = [x for x in os.listdir() if 'xlsx' in x]
file_nm 

['04. GM_IND_MEM_개인회원정보_20220922.xlsx',
 '12. PS_CUST_DAILY_MEMO_PSR 일지(일별 메모)_20220922.xlsx',
 '16. PS_SALE_PSR매출_20220922.xlsx',
 '97.CULT_TR부분접수결제환불.xlsx',
 '03. GM_GRADE_TARGET_회원 등급 최종 대상자_20220921 (PJ추출).xlsx',
 '13. PS_CUST_RESRV_PSR고객예약접수_20220922.xlsx',
 '96.CULT_CUST_MAST문화회원마스터.xlsx',
 '98.CULT_BABY유모차대여.xlsx',
 '99.VIP_WINE상반기 와인 구독 신청 회원번호.xlsx',
 '05. GM_REPRE_MEM_INFO_대표회원정보_20220922.xlsx',
 '15. PS_INVITE_REG_PSR 출입등록 관리_20220922.xlsx',
 '14. PS_FANCY_PSR고객취향_20220922.xlsx',
 '95.PSR 메모 텍스트마이닝 DATA.xlsx',
 'table_schema_info_20221005.xlsx']

* file name 정규화

In [7]:
def reg_file_nm(file_nm):
    
    return re.sub(
        '[_]+$',
        '',
        re.sub(
            '[ㄱ-ㅎ가-힣0-9\.\(\) ]',
            '',
            re.sub(
                '.xlsx',
                '',
                file_nm
            )   
        )
    ) 

* 파일 이름 핸들링 결과

In [8]:
for nm in file_nm: 
	print(reg_file_nm(nm))
	print('-'*100)

GM_IND_MEM_개인회원정보
----------------------------------------------------------------------------------------------------
PS_CUST_DAILY_MEMO_PSR일지일별메모
----------------------------------------------------------------------------------------------------
PS_SALE_PSR매출
----------------------------------------------------------------------------------------------------
CULT_TR부분접수결제환불
----------------------------------------------------------------------------------------------------
GM_GRADE_TARGET_회원등급최종대상자_PJ추출
----------------------------------------------------------------------------------------------------
PS_CUST_RESRV_PSR고객예약접수
----------------------------------------------------------------------------------------------------
CULT_CUST_MAST문화회원마스터
----------------------------------------------------------------------------------------------------
CULT_BABY유모차대여
--------------------------------------------------

* 데이터 불러오기

In [9]:
galleria_db = {}

for nm in file_nm:
    xlsx_dt = pd.read_excel(
        nm,
        None
    )
    sheet_nm = list(xlsx_dt.keys())[0]

    xlsx_df = pd.DataFrame(
        xlsx_dt[sheet_nm]
    )

    galleria_db[reg_file_nm(nm)] = xlsx_df
    
    print(nm)
    print('-'*100)
    
tbl_list = list(galleria_db.keys())

04. GM_IND_MEM_개인회원정보_20220922.xlsx
----------------------------------------------------------------------------------------------------
12. PS_CUST_DAILY_MEMO_PSR 일지(일별 메모)_20220922.xlsx
----------------------------------------------------------------------------------------------------
16. PS_SALE_PSR매출_20220922.xlsx
----------------------------------------------------------------------------------------------------
97.CULT_TR부분접수결제환불.xlsx
----------------------------------------------------------------------------------------------------
03. GM_GRADE_TARGET_회원 등급 최종 대상자_20220921 (PJ추출).xlsx
----------------------------------------------------------------------------------------------------
13. PS_CUST_RESRV_PSR고객예약접수_20220922.xlsx
----------------------------------------------------------------------------------------------------
96.CULT_CUST_MAST문화회원마스터.xlsx
--------------------------------------------------------

* 컬럼명 소문자로 바꾸기 

In [10]:
for tbl_nm in tbl_list:
	galleria_db[tbl_nm].columns = map(str.lower, galleria_db[tbl_nm].columns)

---
* 컬럼에 한글명 붙이기

#### 개인회원 정보 테이블 (GM_IND_MEM) [**base table**]

In [11]:
galleria_db['GM_IND_MEM_개인회원정보'].head()

,mem_no,credit_card_first_join_ymd,memship_join_ymd,foreiner_flag_cd,foreiner_flag_nm,email_recv_agree_yn,sms_agree_yn,parkjade_choice_rsn_cd,parkjade_choice_rsn_nm,dm_recv_agree_yn,tm_agree_yn,dm_addr,dm_office_nm,dm_dept_nm,inet_email_recv_agree_yn,inet_sms_recv_agree_yn
0,12367029,2017-12-02,NaN,1.0,내국인,N,Y,NaN,NaN,Y,N,NaN,NaN,NaN,N,Y
1,12376469,NaN,2017-12-31,4.0,외국인등록증,N,N,NaN,NaN,N,N,NaN,NaN,NaN,N,N
2,12377168,2019-11-09,2018-01-04,1.0,내국인,Y,Y,NaN,NaN,N,N,NaN,NaN,NaN,Y,Y
3,12378037,2018-01-07,NaN,1.0,내국인,Y,Y,NaN,NaN,Y,Y,NaN,NaN,NaN,Y,Y
4,12684462,2022-07-15,2020-01-16,1.0,내국인,N,N,NaN,NaN,N,N,NaN,NaN,NaN,N,N


* Missing 이 80프로 이상이거나, 카테고리가 1가지인 컬럼 Drop 하기

In [12]:
galleria_db['GM_IND_MEM_개인회원정보'].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168328 entries, 0 to 168327
Data columns (total 16 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   mem_no                      168328 non-null  int64  
 1   credit_card_first_join_ymd  124889 non-null  object 
 2   memship_join_ymd            108218 non-null  object 
 3   foreiner_flag_cd            159048 non-null  float64
 4   foreiner_flag_nm            159048 non-null  object 
 5   email_recv_agree_yn         168326 non-null  object 
 6   sms_agree_yn                168326 non-null  object 
 7   parkjade_choice_rsn_cd      1186 non-null    float64
 8   parkjade_choice_rsn_nm      1186 non-null    object 
 9   dm_recv_agree_yn            168295 non-null  object 
 10  tm_agree_yn                 168295 non-null  object 
 11  dm_addr                     5449 non-null    object 
 12  dm_office_nm                4 non-null       object 
 13  dm_dept_nm    

In [13]:
# Drop col 선택 

galleria_db['GM_IND_MEM_개인회원정보'].drop(
	columns = [
		'parkjade_choice_rsn_cd', 
		'parkjade_choice_rsn_nm',
		'dm_addr',
		'dm_office_nm',
		'dm_dept_nm'
	],
	inplace = True	
)

# 데이터 확인 
galleria_db['GM_IND_MEM_개인회원정보'].head()

,mem_no,credit_card_first_join_ymd,memship_join_ymd,foreiner_flag_cd,foreiner_flag_nm,email_recv_agree_yn,sms_agree_yn,dm_recv_agree_yn,tm_agree_yn,inet_email_recv_agree_yn,inet_sms_recv_agree_yn
0,12367029,2017-12-02,NaN,1.0,내국인,N,Y,Y,N,N,Y
1,12376469,NaN,2017-12-31,4.0,외국인등록증,N,N,N,N,N,N
2,12377168,2019-11-09,2018-01-04,1.0,내국인,Y,Y,N,N,Y,Y
3,12378037,2018-01-07,NaN,1.0,내국인,Y,Y,Y,Y,Y,Y
4,12684462,2022-07-15,2020-01-16,1.0,내국인,N,N,N,N,N,N


#### 년도별 회원등급 최종 대상자 PJ 추출 (GM_GRADE_TARGET)

In [14]:
galleria_db['GM_GRADE_TARGET_회원등급최종대상자_PJ추출'].head()

,repre_mem_no,sale_yy,choice_rsn_cd,choice_rsn_nm,tot_net_sale_amt,grade_lar_cd,grade_lar_nm,grade_small_cd,grade_small_nm,mgm_store_cd,mgm_store_nm,concige_voch_use_cnt,pj_apply_yn
0,10138617,2017,1.0,매출기준,24976322.0,2,ParkJade,3,BLUE,1500.0,수원점,0,Y
1,10138629,2017,1.0,매출기준,43348725.0,2,ParkJade,2,WHITE,9100.0,명품관,0,Y
2,10138933,2017,1.0,매출기준,22522817.0,2,ParkJade,3,BLUE,2900.0,센터시티_BLUE,0,Y
3,10139084,2017,1.0,매출기준,31392979.0,2,ParkJade,3,BLUE,9100.0,명품관,0,Y
4,10139248,2017,1.0,매출기준,23455226.0,2,ParkJade,3,BLUE,5410.0,타임월드_BLUE,0,Y


In [15]:
s1 = set(galleria_db['GM_IND_MEM_개인회원정보'].mem_no) 
s2 = set(galleria_db['GM_GRADE_TARGET_회원등급최종대상자_PJ추출'].repre_mem_no)

print('개인회원 정보와 회원등급대상자 ID (FKey) 교집합 개수 :' ,len(s1 & s2),'명')



개인회원 정보와 회원등급대상자 ID (FKey) 교집합 개수 : 28516 명


* 데이터 Join ['GM_IND_MEM_개인회원정보'], ['GM_GRADE_TARGET_회원등급최종대상자_PJ추출']

In [16]:
galleria_df = pd.merge(
	galleria_db['GM_IND_MEM_개인회원정보'],
	galleria_db['GM_GRADE_TARGET_회원등급최종대상자_PJ추출'],
	how = 'left',
	left_on = 'mem_no',
	right_on = 'repre_mem_no'
)

In [17]:
galleria_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208354 entries, 0 to 208353
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   mem_no                      208354 non-null  int64  
 1   credit_card_first_join_ymd  164769 non-null  object 
 2   memship_join_ymd            131836 non-null  object 
 3   foreiner_flag_cd            199068 non-null  float64
 4   foreiner_flag_nm            199068 non-null  object 
 5   email_recv_agree_yn         208352 non-null  object 
 6   sms_agree_yn                208352 non-null  object 
 7   dm_recv_agree_yn            208321 non-null  object 
 8   tm_agree_yn                 208321 non-null  object 
 9   inet_email_recv_agree_yn    208354 non-null  object 
 10  inet_sms_recv_agree_yn      208354 non-null  object 
 11  repre_mem_no                68542 non-null   float64
 12  sale_yy                     68542 non-null   float64
 13  choice_rsn_cd 

---
#### PS 등급 매출 테이블 (PS_SALE_PSR)

In [18]:
galleria_db['PS_SALE_PSR매출'].head()

,mem_no,saldate,store,store_nm,posno,trxnno,seqno,cancflg,cancflg_nm,gretype,...,salrat,itwondate,psr_cust_id,psr_sale_gb,psr_size_grp,psr_size_cd,psr_color_cd,bigo,cdate,udate
0,10614746.0,2017-02-19,9100,명품관,4,380,25,0,정상,0,...,0,2017-02-19,1000015031,NaN,NaN,NaN,NaN,NaN,2017/02/19 20:31:13,NaN
1,10810805.0,2017-07-28,9100,명품관,2603,7,1,0,정상,0,...,0,2017-07-28,20176417,NaN,NaN,NaN,NaN,NaN,2017/07/28 18:48:32,NaN
2,10810805.0,2017-07-28,9100,명품관,2603,7,2,0,정상,0,...,0,2017-07-28,20176417,NaN,NaN,NaN,NaN,NaN,2017/07/28 18:48:32,NaN
3,10343701.0,2017-07-28,9100,명품관,2603,8,1,0,정상,0,...,0,2017-07-28,20143905,NaN,NaN,NaN,NaN,NaN,2017/07/28 18:48:32,NaN
4,10395545.0,2017-07-28,9100,명품관,2603,9,1,0,정상,0,...,0,2017-07-28,1000010913,NaN,NaN,NaN,NaN,NaN,2017/07/28 18:48:32,NaN


In [19]:
galleria_db['PS_SALE_PSR매출'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22598 entries, 0 to 22597
Data columns (total 57 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mem_no        21975 non-null  float64
 1   saldate       22598 non-null  object 
 2   store         22598 non-null  int64  
 3   store_nm      22598 non-null  object 
 4   posno         22598 non-null  int64  
 5   trxnno        22598 non-null  int64  
 6   seqno         22598 non-null  int64  
 7   cancflg       22598 non-null  int64  
 8   cancflg_nm    22598 non-null  object 
 9   gretype       22598 non-null  int64  
 10  gretype_nm    22598 non-null  object 
 11  plcode        22598 non-null  object 
 12  goodcd        22598 non-null  object 
 13  goodcd_nm     4080 non-null   object 
 14  pumbun        22598 non-null  int64  
 15  pumbun_nm     22598 non-null  object 
 16  pumflg        22598 non-null  int64  
 17  pumflg_nm     22598 non-null  object 
 18  pummok_cd     22598 non-nu

In [20]:
print('갤러리아 psr 등급 회원수:' ,len(galleria_db['PS_SALE_PSR매출'].mem_no.unique()))

갤러리아 psr 등급 회원수: 414


* PSR 등급 회원수가 적어 전체 고객에 대한 corr 을 보는 것은 우선 차후에 처리

In [21]:
# 결합 키 데이터 타입 맞추기 (float to int64)

galleria_db['PS_SALE_PSR매출'].mem_no =galleria_db['PS_SALE_PSR매출'].mem_no.astype('Int64')

* Numetric type 컬럼만 추출 

In [22]:

use_col = [ 
	'mem_no',
	'saldate',  
    'salqty',
    'unit', 
    'itdiscrate', 
    'netqty',
    'etcdiscamt', 
    'mobdisc', 
    'cnpcnt',
    'pric',
    'vatamt',
    'salrat'
]

galleria_db['PS_SALE_PSR매출'][use_col].head()


,mem_no,saldate,salqty,unit,itdiscrate,netqty,etcdiscamt,mobdisc,cnpcnt,pric,vatamt,salrat
0,10614746,2017-02-19,120,1900,5,0,0,0,0,1900,19691,0
1,10810805,2017-07-28,0,0,0,0,0,0,0,0,0,0
2,10810805,2017-07-28,1,540000,0,0,0,0,0,0,49091,0
3,10343701,2017-07-28,1,668000,5,0,0,0,0,0,57691,0
4,10395545,2017-07-28,1,1370000,0,0,0,0,0,0,124545,0


* saledate 를 연도별로 split 

In [23]:
galleria_db['PS_SALE_PSR매출'].saldate = galleria_db['PS_SALE_PSR매출'].saldate.str[0:4]


In [24]:
galleria_db['PS_SALE_PSR매출'][use_col].groupby(['mem_no','saldate']).agg(np.average)

salqty          unit  itdiscrate    netqty     etcdiscamt  \
mem_no   saldate                                                                
10023384 2018     1.000000  7.965225e+07    0.000000  0.000000       0.000000   
10039810 2021     1.000000  1.785000e+06    0.000000  0.000000       0.000000   
10045825 2022     1.000000  5.644875e+06    0.000000  0.000000       0.000000   
10046831 2018     1.093458  2.404854e+06    5.355140  0.037383   10274.766355   
         2019     0.884868  3.448581e+06    3.289474  1.394737    2184.868421   
...                    ...           ...         ...       ...            ...   
13020179 2022     1.000000  2.322857e+06    0.000000  0.000000       0.000000   
13229033 2022     1.000000  1.298710e+08    0.000000  0.000000       0.000000   
13270288 2022     2.035714  1.692929e+06    5.714286  0.000000  139285.714286   
13294125 2022     1.000000  2.004517e+07    1.666667  0.000000       0.000000   
13339170 2022     0.833333  1.781667e+06    0.000000  0.000000       0.000000   

                  mobdisc  cnpcnt           pric        vatamt    salrat  
mem_no   saldate                                                          
10023384 2018         0.0     0.0       0.000000  7.241114e+06  0.000000  
10039810 2021         0.0     0.0       0.000000  1.622730e+05  0.000000  
10045825 2022         0.0     0.0       0.000000  5.131704e+05  0.000000  
10046831 2018         0.0     0.0   42037.383178  2.218875e+05  4.392523  
         2019         0.0     0.0    9481.907895  2.676670e+05  3.717105  
...                   ...     ...            ...           ...       ...  
13020179 2022         0.0     0.0       0.000000  2.111687e+05  0.000000  
13229033 2022         0.0     0.0       0.000000  1.180646e+07  0.000000  
13270288 2022         0.0     0.0  465821.428571  1.467652e+05  0.000000  
13294125 2022         0.0     0.0       0.000000  1.735668e+06  0.000000  
13339170 2022         0.0     0.0       0.000000  1.445458e+05  0.000000  

[724 rows x 10 columns]

---

* 데이터 Join ['GM_IND_MEM_개인회원정보','GM_GRADE_TARGET_회원등급최종대상자_PJ추출'], ['PS_SALE_PSR매출']

* 결합 전 ID 교집합 수 확인

In [25]:
s1 = set(galleria_df.mem_no) 
s2 = set(galleria_db['PS_SALE_PSR매출'].mem_no)

print('개인회원 정보와 PS_SALE_PSR(FKey) 교집합 개수 :' ,len(s1 & s2),'명')



개인회원 정보와 PS_SALE_PSR(FKey) 교집합 개수 : 385 명


In [26]:
# 데이터 결합 

galleria_df = pd.merge(
	galleria_df,
	galleria_db['PS_SALE_PSR매출'][use_col].groupby(['mem_no','saldate']).agg(np.average),
	how = 'left',
	left_on = 'mem_no',
	right_on = 'mem_no'
)

---
#### PSR 고객의 각 변수 별 상관관계 Plot

In [27]:
galleria_df.dropna().head()

,mem_no,credit_card_first_join_ymd,memship_join_ymd,foreiner_flag_cd,foreiner_flag_nm,email_recv_agree_yn,sms_agree_yn,dm_recv_agree_yn,tm_agree_yn,inet_email_recv_agree_yn,...,salqty,unit,itdiscrate,netqty,etcdiscamt,mobdisc,cnpcnt,pric,vatamt,salrat
1234,10045825,1994-04-20,2021-12-30,1.0,내국인,N,Y,Y,Y,N,...,1.000000,5.644875e+06,0.000000,0.000000,0.0,0.0,0.0,0.000000,5.131704e+05,0.000000
1235,10045825,1994-04-20,2021-12-30,1.0,내국인,N,Y,Y,Y,N,...,1.000000,5.644875e+06,0.000000,0.000000,0.0,0.0,0.0,0.000000,5.131704e+05,0.000000
1236,10045825,1994-04-20,2021-12-30,1.0,내국인,N,Y,Y,Y,N,...,1.000000,5.644875e+06,0.000000,0.000000,0.0,0.0,0.0,0.000000,5.131704e+05,0.000000
1650,10071807,1994-07-02,2013-11-29,1.0,내국인,N,N,N,N,N,...,2.519481,1.647426e+07,4.350649,4.038961,0.0,0.0,0.0,28509.480519,1.406855e+06,0.000000
1651,10071807,1994-07-02,2013-11-29,1.0,내국인,N,N,N,N,N,...,2.542857,6.105370e+06,6.180952,0.000000,0.0,0.0,0.0,25980.952381,5.671157e+05,1.809524


* Numetric 컬럼 중 사용가능한 컬럼 추출

In [28]:
use_col = [
	'tot_net_sale_amt',
	'grade_lar_cd', 
	'salqty', 
	'unit', 
	'itdiscrate', 
	'netqty', 
	'etcdiscamt', 
	'pric', 
	'vatamt', 
	'salrat'
]

psr_df = galleria_df[use_col].dropna()
psr_df.head()


,tot_net_sale_amt,grade_lar_cd,salqty,unit,itdiscrate,netqty,etcdiscamt,pric,vatamt,salrat
562,277468775.0,1.0,1.0,79652250.0,0.0,0.0,0.0,0.0,7241113.5,0.0
563,106422025.0,1.0,1.0,79652250.0,0.0,0.0,0.0,0.0,7241113.5,0.0
564,349261790.0,2.0,1.0,79652250.0,0.0,0.0,0.0,0.0,7241113.5,0.0
565,32112280.0,2.0,1.0,79652250.0,0.0,0.0,0.0,0.0,7241113.5,0.0
817,61500000.0,2.0,1.0,1785000.0,0.0,0.0,0.0,0.0,162273.0,0.0


In [29]:
# 등급 코드를 역순으로 (psr -> 4 로 바꾸기 (기존은 1로 되어 있음))
psr_df.grade_lar_cd = 5-psr_df.grade_lar_cd

In [30]:
psr_df.rename(
	columns = { 
		'tot_net_sale_amt': '총순매출액',
		'grade_lar_cd': '등급대분류', 
		'concige_voch_use_cnt': '컨시어지 바우처 사용 횟수', 
		'salqty': '평균 매출 수량', 
		'unit': '평균 매가단가', 
		'itdiscrate': '평균 애누리 할인율', 
		'netqty': '평균 내용량', 
		'etcdiscamt': '평균 기타할인금액', 
		'cnpcnt': '평균 구폰건수', 
		'pric': '평균 정상단가', 
		'vatamt': '평균 부가세 금액', 
		'salrat': '평균 행사율'
	},
	inplace=True
)




In [31]:
psr_df

,총순매출액,등급대분류,평균 매출 수량,평균 매가단가,평균 애누리 할인율,평균 내용량,평균 기타할인금액,평균 정상단가,평균 부가세 금액,평균 행사율
562,277468775.0,4.0,1.000000,7.965225e+07,0.000000,0.0,0.000000,0.000000,7.241114e+06,0.000000
563,106422025.0,4.0,1.000000,7.965225e+07,0.000000,0.0,0.000000,0.000000,7.241114e+06,0.000000
564,349261790.0,3.0,1.000000,7.965225e+07,0.000000,0.0,0.000000,0.000000,7.241114e+06,0.000000
565,32112280.0,3.0,1.000000,7.965225e+07,0.000000,0.0,0.000000,0.000000,7.241114e+06,0.000000
817,61500000.0,3.0,1.000000,1.785000e+06,0.000000,0.0,0.000000,0.000000,1.622730e+05,0.000000
...,...,...,...,...,...,...,...,...,...,...
179240,601532549.0,4.0,0.944444,3.395556e+06,0.555556,0.0,0.000000,0.000000,3.014494e+05,3.333333
179241,601532549.0,4.0,1.000000,2.322857e+06,0.000000,0.0,0.000000,0.000000,2.111687e+05,0.000000
195637,0.0,4.0,1.000000,1.298710e+08,0.000000,0.0,0.000000,0.000000,1.180646e+07,0.000000
195638,0.0,4.0,1.000000,1.298710e+08,0.000000,0.0,0.000000,0.000000,1.180646e+07,0.000000


In [32]:

psr_df_corr = psr_df.corr()

psr_df_corr

,총순매출액,등급대분류,평균 매출 수량,평균 매가단가,평균 애누리 할인율,평균 내용량,평균 기타할인금액,평균 정상단가,평균 부가세 금액,평균 행사율
총순매출액,1.000000,0.203954,-0.013531,0.077449,-0.001689,0.030487,-0.009183,0.033523,0.077682,-0.014965
등급대분류,0.203954,1.000000,-0.033046,0.036938,0.029487,-0.021577,0.008185,0.016971,0.036524,-0.062481
평균 매출 수량,-0.013531,-0.033046,1.000000,-0.038070,0.045746,0.005001,0.056361,0.001850,-0.031878,-0.003733
평균 매가단가,0.077449,0.036938,-0.038070,1.000000,-0.219419,-0.027166,-0.013018,0.007937,0.999642,-0.074215
평균 애누리 할인율,-0.001689,0.029487,0.045746,-0.219419,1.000000,0.018175,0.074424,0.028497,-0.218378,-0.007158
평균 내용량,0.030487,-0.021577,0.005001,-0.027166,0.018175,1.000000,-0.002849,0.000032,-0.026882,-0.010074
평균 기타할인금액,-0.009183,0.008185,0.056361,-0.013018,0.074424,-0.002849,1.000000,0.508656,-0.015534,-0.015497
평균 정상단가,0.033523,0.016971,0.001850,0.007937,0.028497,0.000032,0.508656,1.000000,0.006157,-0.016086
평균 부가세 금액,0.077682,0.036524,-0.031878,0.999642,-0.218378,-0.026882,-0.015534,0.006157,1.000000,-0.073916
평균 행사율,-0.014965,-0.062481,-0.003733,-0.074215,-0.007158,-0.010074,-0.015497,-0.016086,-0.073916,1.000000


In [34]:

psr_df_corr = psr_df.corr()

fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        x = psr_df_corr.columns,
        y = psr_df_corr.index,
        z = np.array(psr_df_corr),
        text= psr_df_corr.values,
        texttemplate='%{text:.2f}'
 
    )
)
fig.show()